Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [12]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


1. Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape()

In [16]:
import html

In [44]:
train_df["clear_html"] = train_df.tweet.apply(lambda x: html.unescape(x))
train_df

,id,label,tweet,clear_html
0,1,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","@user #sikh #temple vandalised in in #calgary,..."


2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [45]:
train_df.clear_html = train_df.clear_html.apply(lambda x: re.sub(r'@[\w]*','', x))
train_df.clear_html

0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i can't use cause th...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31957    ate  isz that youuu?ððððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960     #sikh #temple vandalised in in #calgary, #wso...
31961                          thank you  for you follow  
Name: clear_html, Length: 31962, dtype: object

3. Изменим регистр твитов на нижний с помощью .lower()

In [46]:
train_df.clear_html = train_df.clear_html.str.lower()
train_df.clear_html

0          when a father is dysfunctional and is so sel...
1          thanks for #lyft credit i can't use cause th...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31957    ate  isz that youuu?ððððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960     #sikh #temple vandalised in in #calgary, #wso...
31961                          thank you  for you follow  
Name: clear_html, Length: 31962, dtype: object

4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [108]:
def replace_word(sentence, replace_dict):
    return " ".join([replace_dict.get(word, word) for word in sentence.split()])

In [48]:
train_df.clear_html = train_df.clear_html.apply(lambda x: replace_word(x,apostrophe_dict))
train_df

,id,label,tweet,clear_html
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i cannot use cause the...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ur...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation
...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu?ððððððð...
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...","#sikh #temple vandalised in in #calgary, #wso ..."


5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [49]:
train_df.clear_html= train_df.clear_html.apply(lambda x: replace_word(x,short_word_dict))
train_df.clear_html

0        when a father is dysfunctional and is so selfi...
1        thanks for #lyft credit i cannot use cause the...
2                                      bihday your majesty
3        #model i love you take with you all the time i...
4                      factsguide: society now #motivation
                               ...                        
31957    ate isz that youuu?ððððððð...
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    #sikh #temple vandalised in in #calgary, #wso ...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [50]:
train_df.clear_html= train_df.clear_html.apply(lambda x: replace_word(x,emoticon_dict))
train_df.clear_html

0        when a father is dysfunctional and is so selfi...
1        thanks for #lyft credit i cannot use cause the...
2                                      bihday your majesty
3        #model i love you take with you all the time i...
4                      factsguide: society now #motivation
                               ...                        
31957    ate isz that youuu?ððððððð...
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    #sikh #temple vandalised in in #calgary, #wso ...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [51]:
train_df.clear_html = train_df.clear_html.apply(lambda x: re.sub(r'[^\w\s]','', x))
train_df.clear_html

0        when a father is dysfunctional and is so selfi...
1        thanks for lyft credit i cannot use cause they...
2                                      bihday your majesty
3        model i love you take with you all the time in...
4                        factsguide society now motivation
                               ...                        
31957                        ate isz that youuuðððððððððâï
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    sikh temple vandalised in in calgary wso conde...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [52]:
train_df.clear_html = train_df.clear_html.apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
train_df.clear_html

0        when a father is dysfunctional and is so selfi...
1        thanks for lyft credit i cannot use cause they...
2                                      bihday your majesty
3        model i love you take with you all the time in...
4                        factsguide society now motivation
                               ...                        
31957                        ate isz that youuu           
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    sikh temple vandalised in in calgary wso conde...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [53]:
train_df.clear_html = train_df.clear_html.apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
train_df.clear_html

0        when a father is dysfunctional and is so selfi...
1        thanks for lyft credit i cannot use cause they...
2                                      bihday your majesty
3        model i love you take with you all the time in...
4                        factsguide society now motivation
                               ...                        
31957                        ate isz that youuu           
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on a monday morning otw...
31960    sikh temple vandalised in in calgary wso conde...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [58]:
train_df.clear_html = train_df.clear_html.apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
train_df.clear_html

0        when father is dysfunctional and is so selfish...
1        thanks for lyft credit cannot use cause they d...
2                                      bihday your majesty
3          model love you take with you all the time in ur
4                        factsguide society now motivation
                               ...                        
31957                                   ate isz that youuu
31958    to see nina turner on the airwaves trying to w...
31959    listening to sad songs on monday morning otw t...
31960    sikh temple vandalised in in calgary wso conde...
31961                             thank you for you follow
Name: clear_html, Length: 31962, dtype: object

11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [62]:
import nltk

In [63]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kashe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [80]:
train_df["tweet_token"] = train_df.clear_html.apply(lambda x:  nltk.tokenize.word_tokenize(x))
train_df

,id,label,tweet,clear_html,tweet_token
0,1,0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ..."
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"
...,...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"[ate, isz, that, youuu]"
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"[to, see, nina, turner, on, the, airwaves, try..."
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"[listening, to, sad, songs, on, monday, mornin..."
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",sikh temple vandalised in in calgary wso conde...,"[sikh, temple, vandalised, in, in, calgary, ws..."


12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [81]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kashe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
train_df["tweet_token_filtered"] = train_df["tweet_token"].apply(lambda words:  [word for word in words if not word in stop_words])
train_df["tweet_token_filtered"]

0        [father, dysfunctional, selfish, drags, kids, ...
1        [thanks, lyft, credit, use, cause, offer, whee...
2                                        [bihday, majesty]
3                            [model, love, take, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
31957                                    [ate, isz, youuu]
31958    [see, nina, turner, airwaves, trying, wrap, ma...
31959    [listening, sad, songs, monday, morning, otw, ...
31960    [sikh, temple, vandalised, calgary, wso, conde...
31961                                      [thank, follow]
Name: tweet_token, Length: 31962, dtype: object

13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [83]:
stemmer = nltk.stem.PorterStemmer()

In [84]:
train_df["tweet_stemmed"] = train_df["tweet_token"].apply(lambda tokens:  [stemmer.stem(token) for token in tokens])
train_df["tweet_stemmed"]

0        [father, dysfunct, selfish, drag, kid, dysfunc...
1        [thank, lyft, credit, use, caus, offer, wheelc...
2                                        [bihday, majesti]
3                            [model, love, take, time, ur]
4                              [factsguid, societi, motiv]
                               ...                        
31957                                    [ate, isz, youuu]
31958    [see, nina, turner, airwav, tri, wrap, mantl, ...
31959    [listen, sad, song, monday, morn, otw, work, sad]
31960    [sikh, templ, vandalis, calgari, wso, condemn,...
31961                                      [thank, follow]
Name: tweet_stemmed, Length: 31962, dtype: object

14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [98]:
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kashe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [101]:
train_df["tweet_lemmatized"] = train_df["tweet_stemmed"].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
train_df["tweet_lemmatized"]

0        [father, dysfunct, selfish, drag, kid, dysfunc...
1        [thank, lyft, credit, use, caus, offer, wheelc...
2                                        [bihday, majesti]
3                            [model, love, take, time, ur]
4                              [factsguid, societi, motiv]
                               ...                        
31957                                    [ate, isz, youuu]
31958    [see, nina, turner, airwav, tri, wrap, mantl, ...
31959    [listen, sad, song, monday, morn, otw, work, sad]
31960    [sikh, templ, vandalis, calgari, wso, condemn,...
31961                                      [thank, follow]
Name: tweet_lemmatized, Length: 31962, dtype: object

15. Сохраним результат предобработки в pickle-файл.

In [103]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')

In [109]:
def prepare_data(data,               
                 source='tweet',
                 clean ='clean_tweet',
                 token ='tweet_token',
                 filter_data='tweet_token_filtered',
                 stemmed='tweet_stemmed',
                 lemmatized='tweet_lemmatized'):
    data_copy=data
    data_copy[clean] = data_copy[source].apply(lambda x: html.unescape(x))
    data_copy[clean] = data_copy[clean].apply(lambda x: re.sub(r'@[\w]*','', x))
    data_copy[clean] = data_copy[clean].str.lower()
    data_copy[clean] = data_copy[clean].apply(lambda x: replace_word(x, apostrophe_dict))
    data_copy[clean] = data_copy[clean].apply(lambda x: replace_word(x, short_word_dict))
    data_copy[clean] = data_copy[clean].apply(lambda x: replace_word(x, emoticon_dict))
    data_copy[clean] = data_copy[clean].apply(lambda x: re.sub(r'[^\w\s]','', x))
    data_copy[clean] = data_copy[clean].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
    data_copy[clean] = data_copy[clean].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
    data_copy[clean] = data_copy[clean].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
    data_copy[token] = data_copy[clean].apply(lambda x:  nltk.tokenize.word_tokenize(x))
    data_copy[filter_data] = data_copy[token].apply(lambda words:  [word for word in words if not word in stop_words])
    data_copy[stemmed] = data_copy[filter_data].apply(lambda tokens:  [stemmer.stem(token) for token in tokens])
    data_copy[lemmatized] = data_copy[stemmed].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
    return data_copy

In [110]:
train_df_prepare = prepare_data(train_df)
train_df_prepare

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"
...,...,...,...,...,...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"[ate, isz, that, youuu]","[ate, isz, youuu]","[ate, isz, youuu]","[ate, isz, youuu]"
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma...","[see, nina, turner, airwav, tri, wrap, mantl, ...","[see, nina, turner, airwav, tri, wrap, mantl, ..."
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ...","[listen, sad, song, monday, morn, otw, work, sad]","[listen, sad, song, monday, morn, otw, work, sad]"
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",sikh temple vandalised in in calgary wso conde...,"[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde...","[sikh, templ, vandalis, calgari, wso, condemn,...","[sikh, templ, vandalis, calgari, wso, condemn,..."


In [111]:
test_df_prepare = prepare_data(test_df)
test_df_prepare

,id,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic...","[studiolif, aislif, requir, passion, dedic, wi...","[studiolif, aislif, requir, passion, dedic, wi..."
1,31964,@user #white #supremacists want everyone to s...,white supremacists want everyone to see the ne...,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new...","[white, supremacist, want, everyon, see, new, ...","[white, supremacist, want, everyon, see, new, ..."
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your acne altwaystoheal heal...,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health...","[safe, way, heal, acn, altwaystoh, healthi, heal]","[safe, way, heal, acn, altwaystoh, healthi, heal]"
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread...","[hp, curs, child, book, reserv, alreadi, ye, h...","[hp, cur, child, book, reserv, alreadi, ye, ha..."
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",rd bihday to my amazing hilarious nephew eli a...,"[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ...","[rd, bihday, amaz, hilari, nephew, eli, ahmir,...","[rd, bihday, amaz, hilari, nephew, eli, ahmir,..."
...,...,...,...,...,...,...,...
17192,49155,thought factory: left-right polarisation! #tru...,thought factory leftright polarisation trump u...,"[thought, factory, leftright, polarisation, tr...","[thought, factory, leftright, polarisation, tr...","[thought, factori, leftright, polaris, trump, ...","[thought, factori, leftright, polaris, trump, ..."
17193,49156,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverreadi, fo..."
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohioomg used words...,"[hillary, campaigned, today, in, ohioomg, used...","[hillary, campaigned, today, ohioomg, used, wo...","[hillari, campaign, today, ohioomg, use, word,...","[hillari, campaign, today, ohioomg, use, word,..."
17195,49158,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, work, confer, right, mindset, lead, cu...","[happi, work, confer, right, mindset, lead, cu..."


In [115]:
test_df_prepare.to_pickle("output/test_df_processed.pkl")

In [116]:
train_df_prepare.to_pickle("output/train_df_processed.pkl")